## Object Tracking
-  to track objects between two images (or even in a video), you can use OpenCV’s tracking algorithms. This allows you to track moving objects, detect new or missing objects, and visualize their movement.
- Methods for Object Tracking
    - Simple Difference Method → Detects object position changes using cv2.absdiff().
    - Feature Matching (ORB/SIFT) → Tracks objects by identifying key points.
    - Deep Learning-based Tracking (YOLO + SORT or DeepSORT) → Tracks objects persistently across frames.
    - OpenCV Built-in Trackers → Faster object tracking in videos.

###  Method 1: Tracking with Background Subtraction
- This method detects moving objects between two images.

In [ ]:
import cv2
import numpy as np

def track_objects(img1_path, img2_path, output_path="tracked_output.png"):
    # Load images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    # Convert to grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Compute absolute difference
    diff = cv2.absdiff(gray1, gray2)

    # Apply threshold
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Find contours (Moving objects)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw rectangles around detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Ignore small noise
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img2, (x, y), (x+w, y+h), (0, 255, 0), 3)

    # Save the tracked output
    cv2.imwrite(output_path, img2)
    print(f"Object tracking completed. Output saved as {output_path}")

# Run tracking
track_objects("image1.jpg", "image2.jpg")

### Method 2: Object Tracking with OpenCV Trackers (For Video)
- If you want to track an object continuously across frames in a video:

In [ ]:
import cv2

# Initialize tracker (Choose one)
tracker = cv2.TrackerCSRT_create()  # More accurate but slower
# tracker = cv2.TrackerKCF_create()  # Faster but less accurate

# Open video
video = cv2.VideoCapture("video.mp4")

# Read first frame
ret, frame = video.read()
bbox = cv2.selectROI("Select Object", frame, False)
tracker.init(frame, bbox)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Update tracker
    success, bbox = tracker.update(frame)

    if success:
        x, y, w, h = [int(a) for a in bbox]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow("Tracking", frame)

    if cv2.waitKey(20) & 0xFF == 27:  # Press ESC to exit
        break

video.release()
cv2.destroyAllWindows()

### Method 3: YOLO + SORT (Best for Object Tracking in Images & Videos)
-  If you need accurate multi-object tracking, you can use YOLO (for detection) + SORT/DeepSORT (for tracking across frames).

In [ ]:
from ultralytics import YOLO
from sort import Sort  # SORT tracker library

# Load YOLOv8 Model
model = YOLO("yolov8n.pt")

# Initialize SORT tracker
tracker = Sort()

# Read image
img = cv2.imread("image1.jpg")

# Detect objects using YOLO
results = model(img)

# Convert results to bounding boxes
detections = []
for r in results:
    for box in r.boxes:
        x1, y1, x2, y2, score, cls = box.xyxy.tolist()[0]
        detections.append([x1, y1, x2, y2, score])

# Update tracker
tracks = tracker.update(np.array(detections))

# Draw tracked objects
for track in tracks:
    x1, y1, x2, y2, track_id = [int(x) for x in track]
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, f"ID {track_id}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imwrite("tracked_output.jpg", img)
print("Object tracking complete.")

###  Which Tracking Method to Use?
### Method	Best For	Pros	Cons
- Simple Difference (cv2.absdiff)	Static image changes	Easy, Fast	Cannot track moving objects
- OpenCV Trackers	Tracking a single object in video	Works on real-time video	Not good for multiple objects
- YOLO + SORT	Tracking multiple objects	Best for real-world tracking	Requires more processing power

### Motion Heatmap (Tracks Object Movement Over Time)
- A motion heatmap shows where an object has moved in a video by accumulating changes over frames.
- If you want motion heatmaps and trajectory tracking, we can enhance object tracking by visualizing movement patterns over time.
- Output: motion_heatmap.jpg will show where objects moved most in the video.

In [ ]:
import cv2
import numpy as np

# Open the video file
video = cv2.VideoCapture("video.mp4")

# Get frame dimensions
ret, first_frame = video.read()
height, width = first_frame.shape[:2]

# Create an empty heatmap (black image)
heatmap = np.zeros((height, width), dtype=np.float32)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Accumulate movement by adding pixel intensities
    heatmap += gray.astype(np.float32)

# Normalize heatmap (Scale values between 0-255)
heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
heatmap = np.uint8(heatmap)

# Apply a heatmap color mapping
colored_heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Save heatmap image
cv2.imwrite("motion_heatmap.jpg", colored_heatmap)

print("Motion heatmap saved as motion_heatmap.jpg")

###  Object Trajectory Tracking (Using OpenCV & Centroid Tracking)
-  This tracks object paths by connecting their movement across frames.

In [ ]:
mport cv2
import numpy as np

# Open video
video = cv2.VideoCapture("video.mp4")

# Background subtractor for motion detection
fgbg = cv2.createBackgroundSubtractorMOG2()

# Store object trajectory points
trajectory_points = []

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction
    fgmask = fgbg.apply(gray)

    # Find contours (Moving objects)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter small objects
            x, y, w, h = cv2.boundingRect(contour)
            centroid = (x + w // 2, y + h // 2)
            trajectory_points.append(centroid)

            # Draw bounding box and centroid
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, centroid, 5, (0, 0, 255), -1)

    # Draw trajectory lines
    for i in range(1, len(trajectory_points)):
        cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (255, 0, 0), 2)

    cv2.imshow("Trajectory Tracking", frame)
    if cv2.waitKey(30) & 0xFF == 27:  # Press ESC to exit
        break

video.release()
cv2.destroyAllWindows()

### Advanced: Multi-Object Trajectory Tracking (YOLO + DeepSORT)
- For complex tracking, use YOLOv8 + DeepSORT to track multiple objects and assign unique IDs.
-  Python Code (Using YOLO + DeepSORT)

In [ ]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2

# Load YOLO model
model = YOLO("yolov8n.pt")

# Initialize DeepSORT tracker
tracker = DeepSort()

# Open video
video = cv2.VideoCapture("video.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Detect objects using YOLO
    results = model(frame)
    detections = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2, score, cls = box.xyxy.tolist()[0]
            detections.append(([x1, y1, x2 - x1, y2 - y1], score, int(cls)))

    # Track objects
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if track.is_confirmed():
            x, y, w, h = track.to_ltrb()
            track_id = track.track_id
            centroid = (int(x + w // 2), int(y + h // 2))

            # Draw bounding box & trajectory
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
            cv2.putText(frame, f"ID {track_id}", (int(x), int(y) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, centroid, 5, (0, 0, 255), -1)

    cv2.imshow("Multi-Object Tracking", frame)
    if cv2.waitKey(30) & 0xFF == 27:
        break

video.release()
cv2.destroyAllWindows()

### Which Tracking Method Should You Use?
- Method	Use Case	Pros	Cons
- Motion Heatmap (cv2.absdiff)	Detect areas with high movement	Easy to use	Doesn’t track objects individually
- Centroid Tracking	Track a single moving object	Simple & fast	Doesn’t work for multiple objects
- YOLO + DeepSORT	Multi-object tracking in real time	Best for multiple objects	Requires more processing power

### More
-  bounding box tracking over time